Declare some basic symbols:
A_v denotes a variable A that is a vector
Capital letter denote Group points
Small letters denote group scalars

polynomial terms are also used in `X`, `T`, but they would be clear from context

In [202]:
from sympy import *
v, G = MatrixSymbol('v', 1, 1), MatrixSymbol('G', 1, 1)
h_n = 5
g_n = 4
gamma_v = MatrixSymbol('gamma_v', 1, h_n)
G_v = MatrixSymbol('G_v', g_n, 1)
H_v = MatrixSymbol('H_v', h_n, 1)
# T value commitment
# gamma_v scalar is multiplies to H_0, the first term of H_v. Others are 0
V = v*G + gamma_v*H_v
print(V)

gamma_v*H_v + v*G


Define
    - d_v(vector of digits of v),
    - m_v(vector of multiplicities of digits of v)
    - r_v(vector of reciprocals of v)
    - s(vector of blinding factors)

We will first attempt at dealing with the last verification equation in terms of T

In [203]:
from sympy.solvers.solveset import linear_coeffs
d_v, m_v, r_v, s_v = MatrixSymbol('d_v', 1, g_n), MatrixSymbol('m_v', 1, g_n), MatrixSymbol('r_v', 1, g_n), MatrixSymbol('s_v', 1, g_n)
ld_v, lm_v, lr_v, ls_v = MatrixSymbol('ld_v', 1, h_n), MatrixSymbol('lm_v', 1, h_n), MatrixSymbol('lr_v', 1, h_n), MatrixSymbol('ls_v', 1, h_n)
D = d_v*G_v + ld_v*H_v
M = m_v*G_v + lm_v*H_v
R = r_v*G_v + lr_v*H_v
S = s_v*G_v + ls_v*H_v

# create w_v
T = symbols('T', commutative=True)

# Consider the expression
C = S + T*M + T**2*D + T**3*R + T**5*V
C = expand(C)
w_v = linear_coeffs(C, G_v)[0]
l_v = linear_coeffs(C, H_v)[0]
v_hat = linear_coeffs(C, G)[0]

print("Coeffs of G_v:", w_v)
print("Coeffs of H_v:", l_v)
print("Coeffs of G:", v_hat)

Coeffs of G_v: T**3*r_v + T**2*d_v + T*m_v + s_v
Coeffs of H_v: T**5*gamma_v + T**3*lr_v + T**2*ld_v + T*lm_v + ls_v
Coeffs of G: T**5*v


Norm argument:
$|\vec{w}|^2_q + \langle \vec{l},\vec{c} \rangle = v$ for a given $C = vG + \langle w, \vec{G} \rangle + \langle l,\vec{H} \rangle$

In [213]:
import sympy.vector
c_v = Matrix([[1], [T], [T**2], [T**3], [T**4]])

# There should be a better way to do this. Wait response from stack exchange
manual_dot = l_v[0]*c_v[0]
for i in range(1, len(c_v)):
    manual_dot += l_v[i]*c_v[i]

expr = w_v*transpose(w_v) + l_v*c_v + 2*T**5*v
expanded_expr = expand((w_v*transpose(w_v))[0] + manual_dot - 2*T**5*v[0])

# print(expr)

print(expanded_expr.coeff(T, 5))




(2*T**5)*v + (T**3*r_v + T**2*d_v + T*m_v + s_v)*(T**3*r_v.T + T**2*d_v.T + T*m_v.T + s_v.T) + (T**5*gamma_v + T**3*lr_v + T**2*ld_v + T*lm_v + ls_v)*Matrix([
[   1],
[   T],
[T**2],
[T**3],
[T**4]])
2*d_v[0, 0]*r_v[0, 0] + 2*d_v[0, 1]*r_v[0, 1] + 2*d_v[0, 2]*r_v[0, 2] + 2*d_v[0, 3]*r_v[0, 3] + gamma_v[0, 0] + ld_v[0, 3] + lm_v[0, 4] + lr_v[0, 2] - 2*v[0, 0]


We see that the output for coefficient for `T**5` is
```
2*d_v[0, 0]*r_v[0, 0] + 2*d_v[0, 1]*r_v[0, 1] + 2*d_v[0, 2]*r_v[0, 2] + 2*d_v[0, 3]*r_v[0, 3] + gamma_v[0, 0] + ld_v[0, 3] + lm_v[0, 4] + lr_v[0, 2] - 2*v[0, 0]
```
<!-- Rearranging it, we get: $2(\vec{d} \dot \vec{r} - v)$
+ gamma_v[0, 0] + ld_v[0, 3] + lm_v[0, 4] + lr_v[0, 2] -->